In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries
# Libraries for calculation
import pandas as pd
import numpy as np
from scipy import stats
import calendar
import datetime
#Libraries for Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Libraries for NLP
import nltk
import re
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('wordnet')
#Supress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Import comcast telecom complaints data and make a working copy
original_dataset = pd.read_csv("../input/comcastcomplaints/comcast_fcc_complaints_2015.csv")
df = original_dataset[:]

# **Question 1**
Import comcast telecom complaints data to Python

In [ ]:
#Check for datatype of columns
df.dtypes

In [ ]:
#Check for null values
df[df.isnull()].count()

In [ ]:
#convert date variables to datetime.
df['Date'] = pd.to_datetime(df['Date'])
#df['Date_month_year'] = pd.to_datetime(df['Date_month_year'])
#variable for month and year
df['Mon_Year'] = df['Date'].dt.strftime('%b-%y')
#variable for day of week - name and number
df['day_of_week'] = df['Date'].dt.strftime('%A')
df['nday_of_week'] = df['Date'].dt.strftime('%w')
#variable for date in month
df['day_of_mon'] = df['Date'].dt.strftime('%d')
#variable for just date in DD-MM-YY format without timestamp
df['Date_var'] = df['Date'].dt.strftime('%d-%b-%y')
#sort dataframe by Date_var
df = df.sort_values(by = 'Date_var')

# **Question 2**
Provide the trend chart for the number of complaints at monthly and daily granularity levels.

In [ ]:
#Pie plot for number of complaints by month
df['Mon_Year'].value_counts().plot.pie(autopct='%1.1f%%')

The complaints are from April to June. June 2015 has most number of complaints, 57.6% of the total complaints
on the dataset.

In [ ]:
#sort by weekday
df = df.sort_values(by = 'nday_of_week')
#Bar plot for number of complaints weekday
fig , ax = plt.subplots(figsize=(15,7))
plot = sns.countplot(df['day_of_week'])
ticks = plt.setp(plot.get_xticklabels(), rotation=90)

More complaints are recieved midweek, around Tuesday and Wednesday. Least complaints on Sundays.
Customer prefer to complaint during middle of the week and coverage to address them should be maximum
around midweek.

In [ ]:
#sort by date in month
df = df.sort_values(by = 'day_of_mon')
#Bar plot for number of complaints by date in month
fig , ax = plt.subplots(figsize=(15,7))
plot = sns.countplot(df['day_of_mon'])
ticks = plt.setp(plot.get_xticklabels(), rotation=90)

Complaints are almost steady across most dates in the month except for 23rd and 24th. Only May had 31 days
in it and so less data on that date. The increase in complaints for 23rd and 24th are explored with next graph.

In [ ]:
#Sort by date
df = df.sort_values(by='Date')
#Bar plot for number of complaints by date
fig , ax = plt.subplots(figsize=(15,7))
plot = sns.countplot(df['Date_var'])
ticks = plt.setp(plot.get_xticklabels(), rotation=90)

On June 23, 24 and 25th the complaints are unusually more than for other dates. There might be operational
issues which caused the hike.

# **Question 3**
Provide a table with the frequency of complaint types. Which complaint types are maximum i.e., around internet,
network issues, or across any other domains.

In [ ]:
#function to clean up text
def cleanup_lem(row):
    row = row.lower() #covert text to lower
    row = re.sub('[^a-zA-Z0-9]', " ", row) #remove punctuations
    row = row.split() #Tokenization
    #Lemmatize the text and remove stopwords
    lem = WordNetLemmatizer()
    row = [lem.lemmatize(word) for word in row if not word in set(stopwords.words('english'))]
    row = ' '.join(row) #join words again
    return row

In [ ]:
#New column after customer complaints are processed with function to clean up and lemmatize.
df['clean_complaints']=df['Customer Complaint'].apply(lambda x : cleanup_lem(x))

In [ ]:
#Build complaint type based on keywords
#build complaint type as others for all the records
df['complaint_type'] = 'Others'
#build complaint type as account based on keywords
cond2 = (df['clean_complaints'].str.contains('account'))|(df['clean_complaints'].str.contains('service'))|(df['clean_complaints'].str.contains('bill'))|(df['clean_complaints'].str.contains('customer'))|(df['clean_complaints'].str.contains('comcast'))|(df['clean_complaints'].str.contains('business'))|(df['clean_complaints'].str.contains('cancellation'))|(df['clean_complaints'].str.contains('package'))
df.loc[cond2, 'complaint_type'] = 'Account'
#build complaint type as network based on keywords
cond1 = (df['clean_complaints'].str.contains('network'))|(df['clean_complaints'].str.contains('bandwith'))|(df['clean_complaints'].str.contains('speed'))|(df['clean_complaints'].str.contains('connect'))|(df['clean_complaints'].str.contains('disconnect'))|(df['clean_complaints'].str.contains('disruption'))|(df['clean_complaints'].str.contains('outage'))|(df['clean_complaints'].str.contains('reception'))
df.loc[cond1, 'complaint_type'] = 'Network'
#build complaint type as internet based on keywords
cond = (df['clean_complaints'].str.contains('internet'))|(df['clean_complaints'].str.contains('data'))|(df['clean_complaints'].str.contains('online'))|(df['clean_complaints'].str.contains('cap'))
df.loc[cond, 'complaint_type'] = 'Internet'

In [ ]:
#review complaints and complaint type
df[['Customer Complaint','clean_complaints','complaint_type']].head(20)

In [ ]:
# Table of complaint types and counts
complaints_type_count = df.groupby(['complaint_type']).size().sort_values(ascending=False).to_frame().reset_index().rename({0: "count"}, axis=1)
complaints_type_count.sort_values(by=['count'])

In [ ]:
#Bar plot for number of complaint type count
fig , ax = plt.subplots(figsize=(15,7))
plot = sns.countplot(df['complaint_type'])
ticks = plt.setp(plot.get_xticklabels(), rotation=90)

Most of the complaints are under categories of account and internet. To note that the word service is identified
as account.

In [ ]:
#Analyze what causes high number of complaints between June 23-25.
#Subset dataframe with data between June 23-25
Dateswithmaxcomplaints = ['23-Jun-15','24-Jun-15','25-Jun-15']
Jun23_25complaints = df.loc[df['Date_var'].isin(Dateswithmaxcomplaints)]

In [ ]:
Jun23_25complaints

In [ ]:
# Table of complaint types and counts in June 23-25
Jun23_25complaints_type_count = Jun23_25complaints.groupby(['complaint_type']).size().sort_values(ascending=False).to_frame().reset_index().rename({0: "count"}, axis=1)
Jun23_25complaints_type_count.sort_values(by=['count'])

Most of the complaints are under categories of account and internet, similar to larger dataset. To note that the
word service is identified as account. We can assume the opertaion was bad that hike, there was no different
pattern compared to the larger dataset.

# **Question 4**
Create a new categorical variable with value as Open and Closed. Open & Pending is to be categorized as Open
and Closed & Solved is to be categorized as Closed.

In [ ]:
#Values of Status Column
df['Status'].unique()
#Status categorized to Open and Closed.
df['NewStatus'] = ["Open" if Status=="Open" or Status=="Pending" else "Closed" for Status in df['Status']]
df['NewStatus'].unique()

# **Question 5**
Provide state wise status of complaints in a stacked bar chart. Use the categorized variable from Q4. Provide
insights on: Which state has the maximum complaints Which state has the highest percentage of unresolved
complaints

In [ ]:
#State with maximum complaints
complaints_count_state = df.groupby(['State']).size().sort_values(ascending=False).to_frame().reset_index().rename({0: "Count"}, axis=1)
complaints_count_state.sort_values(by=['State'])

In [ ]:
#Bar plot for number of complaints by state
fig , ax = plt.subplots(figsize=(15,7))
plot = sns.barplot(x=complaints_count_state['State'],y=complaints_count_state['Count'],order=complaints_count_state.sort_values('State').State)
ticks = plt.setp(plot.get_xticklabels(), rotation=90)

States with more than 200 complaints CA, FL and GE. States with more than 150 complaints CA, FL, GE and IL
States with more than 100 complaints CA, FL, GE, IL, MI, PA, TN and WA. States with more than 50 complaints
CA, FL, GE, IL, MI, PA, TN, WA, CO, IN, MD, MA, NJ, TX and VA.
Larger states like NY, AZ and NC have performed better compared to other large states. Operations should
improve in GE, FL and CA.

In [ ]:
#Bar plot of complaints and status by state
fig , ax = plt.subplots(figsize=(15,7))
plot1= sns.countplot('State',data=df,hue='NewStatus')
ticks = plt.setp(plot1.get_xticklabels(), rotation=90)

GE, CA and TN have more open issues. Issues are addressed and closed better comparatively in FL.

In [ ]:
#stacked bar chart of statys complaint status
df_plot = df.groupby(['State', 'NewStatus']).size().reset_index().pivot(columns='NewStatus', index='State', values=0)
df_plot.plot(kind='bar', stacked=True, figsize=(15,7))

In [ ]:
#State has the maximum complaints
complaints_count_state.sort_values(by=['Count'],ascending=False)[:5]
print(complaints_count_state[complaints_count_state['Count'] == complaints_count_state['Count'].max()])

In [ ]:
#Which state has the highest percentage of unresolved complaints
State_complaints = df.groupby(['State','NewStatus']).size().unstack().fillna(0)
State_complaints['Total'] = State_complaints['Closed']+State_complaints['Open']
State_complaints['Unresolved%'] = State_complaints['Open']/State_complaints['Total']
State_complaints.round(2)
print(State_complaints[State_complaints['Unresolved%'] == State_complaints['Unresolved%'].max()])

Georgia has most complaints among the states and Kansas has highest unresolved percentage of issues.
However, Kansas has total of 2 cases, 1 open and another closed.

# **Question 6**
Provide the percentage of complaints resolved till date, which were received through the Internet and customer
care calls.

In [ ]:
#Resolved complaints and channel through which complaint was raised.
Complaints_channel = df.groupby(['Received Via','NewStatus']).size().unstack().fillna(0)
Complaints_channel ['Total'] = Complaints_channel['Closed']+Complaints_channel['Open']
Complaints_channel['Resolved%'] = Complaints_channel['Closed']/Complaints_channel['Total']
Complaints_channel

The number of issues closed and open are similar across the channels through which complaints are received.